In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [2]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver  
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys  
from bs4 import BeautifulSoup
from time import sleep
import datetime

In [31]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36")
browser = webdriver.Chrome('../../../chromedriver', chrome_options=opts)
browser.set_page_load_timeout(30)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
seloger = 'https://www.seloger.com/annonces/achat/appartement/paris-11eme-75/bastille-popincourt/162206091.htm?projects=2&types=1&natures=1,2&places=[{cp:75}|{ci:940080}|{ci:940067}]&rooms=1,2,3&enterprise=0&qsVersion=1.0&m=search_to_detail'

In [5]:
browser.get('https://www.seloger.com')
browser.get(seloger)  
sleep(5)

In [7]:
html_source = browser.page_source
property_soup = BeautifulSoup(html_source, 'html.parser')
url = browser.current_url

In [8]:
main_tag = property_soup.find_all('div', class_=re.compile("^app__CWrapMain-aroj7e-3"))[0]
summary_block = main_tag.find_all('div', class_=re.compile("^Summarystyled__TagsWrapper-tzuaot-19"))[0]
quartier_block = main_tag.find('p', class_=re.compile("^Map__AddressLine-sc-6i077b-2"))
price_block = main_tag.find('section', attrs={'data-test': 'price-block'})
description_block = \
    (main_tag.find_all('div', class_=re.compile('^TitledDescription__TitledDescriptionContent-sc-1r4hqf5-1')))
diagnostics_block = main_tag.find_all('div', id='diagnostics')[0]

### Meta block

In [9]:
title = property_soup.title.string
url = property_soup.find('meta', attrs={'property': 'og:url'}).get('content')
dimensions_block = (
    summary_block
        .find_all('div', class_=re.compile("^TagsWithIcon__TagContainer-j1x9om-2"))
)
meta = []
for item in dimensions_block:
    meta.append(item.find_all('div')[-1].string)

In [10]:
(title, url, meta)

('Vente Appartement 2 pièces Paris 11ème - Appartement F2/T2/2 pièces 48m² 598000€ - SeLoger',
 'https://www.seloger.com/annonces/achat/appartement/paris-11eme-75/bastille-popincourt/162206091.htm',
 ['2 pièces', '1 chambre', '48m²', '12 458 € / m²'])

### Price block

In [12]:
current_price = price_block.find_all('div',
                                     class_=re.compile('^Pricestyled__Price-uc7t2j-0'))[0].find('div').string
try:
    deposit_block = price_block.find_all('div', class_=re.compile('^rentHelper__Garantie-sc-1x3dozo-0'))[0]
    deposit = deposit_block.strong.contents[0]
except IndexError:
    deposit = 'N/A'
try:
    price_fluctuations_block = \
        price_block.find_all('div', class_=re.compile('^PriceHistorystyled__Container-sc-18jhpbr-0'))[0]
    price_change_text = ''.join(price_fluctuations_block.find_all('div', class_=re.compile(
        '^PriceHistorystyled__FirstEvolutionFull-sc-18jhpbr-2'))[0].contents[:3])
    price_change_amount = (
        price_fluctuations_block
            .find_all('span',
                      class_=re.compile('^PriceHistorystyled__BoldDisplayAmount-sc-18jhpbr-4'))
    )[0].contents[0]
    price_change_text += price_change_amount
except IndexError:
    price_change_text = 'N/A'

In [13]:
(current_price, deposit, price_change_text)

('598 000 €', 'N/A', 'Baisse  du prix de 27 000 €')

### Geo information

In [14]:
city = ''.join(quartier_block.strong.contents[2:])
neighborhood = quartier_block.contents[-3]

In [15]:
city, neighborhood

('Paris  ( 75011 )', 'Bastille-Popincourt')

### Amenities

In [16]:
try:
    if len(description_block) == 5 and 'Diagnostics' in description_block[-1].find('div').get('class')[0]:
        general = description_block[-4].find_all('ul', class_=re.compile('^GeneralList__List-sc-9gtpjm-0'))[0].find_all(
            'li')
        inside = description_block[-3].find_all('ul', class_=re.compile('^GeneralList__List-sc-9gtpjm-0'))[0].find_all('li')
        other = description_block[-2].find_all('ul', class_=re.compile('^GeneralList__List-sc-9gtpjm-0'))[0].find_all('li')
        general_list = [s.string for s in general]
        inside_list = [s.string for s in inside]
        other_list = [s.string for s in other]
    elif len(description_block) == 4 and 'Diagnostics' in description_block[-1].find('div').get('class')[0]:
        general = description_block[-4].find_all('ul', class_=re.compile('^GeneralList__List-sc-9gtpjm-0'))[0].find_all(
            'li')
        inside = description_block[-3].find_all('ul', class_=re.compile('^GeneralList__List-sc-9gtpjm-0'))[0].find_all('li')
        other = description_block[-2].find_all('ul', class_=re.compile('^GeneralList__List-sc-9gtpjm-0'))[0].find_all('li')
        general_list = [s.string for s in general]
        inside_list = [s.string for s in inside]
        other_list = [s.string for s in other]
    else:
        general_list = []
        inside_list = []
        other_list = []
except IndexError: # No other field
    general = description_block[-3].find_all('ul', class_=re.compile('^GeneralList__List-sc-9gtpjm-0'))[0].find_all(
        'li')
    inside = description_block[-2].find_all('ul', class_=re.compile('^GeneralList__List-sc-9gtpjm-0'))[0].find_all('li')
    general_list = [s.string for s in general]
    inside_list = [s.string for s in inside]
    other_list = []

In [17]:
general_list, inside_list, other_list

([], [], [])

In [19]:
len(description_block)

6

In [20]:
description_block[0]

<div class="TitledDescription__TitledDescriptionContent-sc-1r4hqf5-1 koqVoo"><div><div class="ShowMoreText__UITextContainer-sc-5ggbbc-0 hCeOyd" style="max-height: 66px;"><p>Paris 11ème - rue des TAILLANDIERS: situé à proximité de la place de la Bastille et de la mairie du 11ème, ce 2 pièces de 48 m² se situe dans un immeuble de standing avec ascenseur (code) de 1998 sécurisé, bien entretenu et qui propose de belles prestations. L'appartement CLAIR est situé au 3ème étage sur 4. Il se compose d'une entrée avec rangements, d'un salon/séjour de 22 m², une cuisine américaine équipée et aménagée. Cet espace de vie fait 30 m². D'un coin nuit. Le coin nuit se compose d'une chambre avec rangements, d'une salle de bain avec WC. Une cave complète le bien. L'appartement est en très bon état. Possibilité d'un emplacement de parking en SUS au sous-sol de l'immeuble (30 000 euros) LES POINTS +: PRESTATIONS - LOCALISATION - DISTRIBUTION - CLAIR Copropriété de 25 lots. Charges annuelles: 1800.00 euros

In [21]:
description_block[1]

<div class="TitledDescription__TitledDescriptionContent-sc-1r4hqf5-1 koqVoo"><div class="PlusList__Container-m3s89e-0 ipbCnI" data-id="1" data-test="plus-list"><figure class="Plus__Figure-sc-1v5witi-0 iGqmPS"><div class="Plus__ImageContainer-sc-1v5witi-1 bhuRZQ"></div><figcaption>Cave</figcaption></figure><figure class="Plus__Figure-sc-1v5witi-0 iGqmPS"><div class="Plus__ImageContainer-sc-1v5witi-1 bhuRZQ"></div><figcaption>Ascenseur</figcaption></figure></div></div>

In [22]:
description_block[2]

<div class="TitledDescription__TitledDescriptionContent-sc-1r4hqf5-1 koqVoo"><ul class="GeneralList__List-sc-9gtpjm-0 BAyYz"><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Surface de 48m²</li><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Bâtiment de 5 étages</li><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">2 Pièces</li><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">1 Chambre</li><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Au 3ème étage</li></ul></div>

In [23]:
description_block[3]

<div class="TitledDescription__TitledDescriptionContent-sc-1r4hqf5-1 koqVoo"><ul class="GeneralList__List-sc-9gtpjm-0 BAyYz"><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">1 Salle de bain</li><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">1 Toilette</li><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Rangements</li><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Chauffage individuel électrique radiateur</li><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Cuisine aucune</li><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Une entrée</li><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Salle de séjour</li></ul></div>

In [24]:
description_block[4]

<div class="TitledDescription__TitledDescriptionContent-sc-1r4hqf5-1 koqVoo"><ul class="GeneralList__List-sc-9gtpjm-0 BAyYz"><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Interphone</li><li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Digicode</li></ul></div>

In [25]:
description_block[5]

<div class="TitledDescription__TitledDescriptionContent-sc-1r4hqf5-1 koqVoo"><div class="Diagnostics__DiagnosticsContainer-al64ti-2 giLtwp" data-test="diagnostics-content"><div class="Diagnostics__PreviewContainer-al64ti-0 dTGPCg"><p class="Diagnostics__PreviewTitle-al64ti-1 eJGktk" data-test="diagnostics-preview-title">Diagnostic de performance énergétique</p><div class="Preview__PreviewBar-sc-1pa12ii-0 jIpQSL" data-test="diagnostics-preview-bar"><div class="Preview__PreviewTile-sc-1pa12ii-1 gafcOY"><p>A</p></div><div class="Preview__PreviewTile-sc-1pa12ii-1 iGmEzL"><p>B</p></div><div class="Preview__PreviewTile-sc-1pa12ii-1 edGcMP"><p>C</p></div><div class="Preview__PreviewTile-sc-1pa12ii-1 Preview__PreviewFocusedTile-sc-1pa12ii-2 hJiFNB"><p>D</p><div class="Preview__PreviewTooltip-sc-1pa12ii-3 hVDlNM"><span class="Preview__PreviewTooltipValue-sc-1pa12ii-4 eGDcBO">224</span><span class="Preview__PreviewTooltipCaption-sc-1pa12ii-5 cizigk">kWhEP/m².an</span></div></div><div class="Prev

### Energy

In [27]:
try:
    [energy_diagnostic, ges_diagnostics] = diagnostics_block.find_all('div',
                                                                      class_=re.compile(
                                                                          '^Diagnostics__DiagnosticsContainer-al64ti-2'))
    letter = energy_diagnostic.find_all('div', class_=lambda x: x and 'FocusedTile' in x)[0].p.string
    cons = energy_diagnostic.find('span', class_=re.compile('^Preview__PreviewTooltipValue-sc-1pa12ii-4')).string
    cons += energy_diagnostic.find('span', class_=re.compile('^Preview__PreviewTooltipCaption-sc-1pa12ii-5')).string
except IndexError:
    letter = 'N/A'
    cons = 'N/A'
except AttributeError:
    letter = 'N/A'
    cons = 'N/A'

In [28]:
letter, cons

('D', '224kWhEP/m².an')

### List properties

In [32]:
browser.get('https://www.seloger.com/list.htm?projects=2&types=1&natures=1%2C2&places=%5B%7Bcp%3A75%7D%7C%7Bcp%3A93%7D%7C%7Bcp%3A91%7D%7C%7Bcp%3A94%7D%7C%7Bcp%3A92%7D%5D&rooms=1%2C2%2C3&enterprise=0&qsVersion=1.0&LISTING-LISTpg=2')  

In [33]:
html_source = browser.page_source
sr_soup = BeautifulSoup(html_source, 'html.parser')

In [66]:
property_urls = []
main_tag = sr_soup.find_all('div', class_='Page__WrapMain-st6q56-1 bgOcDb')[0]
results = main_tag.find_all('div', class_="block__ShadowedBlock-sc-10w6hsj-0")
for result in results:
    try:
        property_urls.append(result.find_all('a', class_='CoveringLink-a3s3kt-0 dXJclF')[0].get('href'))
    except IndexError:
        pass

In [69]:
len(property_urls)

25

In [ ]:
<div class="block__ShadowedBlock-sc-10w6hsj-0 ListContent__SmartClassifiedExtended-sc-1viyr2k-1 djVfZb classified__ClassifiedContainer-sc-1wmlctl-0 jzFgwH Card__CardContainer-sc-7insep-7 iyJAUQ"><div class="Card__MediaZone-sc-7insep-0 fPingW"><div class="PhotoSlider__PhotoSliderContainer-cta2l8-0 feqESw"><div data-testid="gsl.uilib.Slider" direction="1" class="sc-jWBwVP cUnYXI"><div hidden=""><div><div><div class="Card__Photo-sc-7insep-2 dIgcyT" style="background-image: url(&quot;https://v.seloger.com/s/width/400/visuels/0/p/n/1/0pn1x6e2kwux6ed7o345iylenz7i1qlc4mczvnnrz.jpg&quot;);"></div><a z="1" href="https://www.seloger.com/annonces/achat/appartement/vincennes-94/162414425.htm?projects=2&amp;types=1&amp;natures=1,2&amp;places=[{cp:75}|{cp:93}|{cp:91}|{cp:94}|{cp:92}]&amp;rooms=1,2,3&amp;enterprise=0&amp;qsVersion=1.0&amp;m=search_to_detail" target="_blank" class="CoveringLink-a3s3kt-0 dXJclF"></a></div></div><div><div><div class="Card__Photo-sc-7insep-2 dIgcyT" style="background-image: url(&quot;https://v.seloger.com/s/width/400/visuels/2/0/1/2/201279niip8kdlxz6s8nc61b19586u3okz4fz1ycf.jpg&quot;);"></div><a z="1" href="https://www.seloger.com/annonces/achat/appartement/vincennes-94/162414425.htm?projects=2&amp;types=1&amp;natures=1,2&amp;places=[{cp:75}|{cp:93}|{cp:91}|{cp:94}|{cp:92}]&amp;rooms=1,2,3&amp;enterprise=0&amp;qsVersion=1.0&amp;m=search_to_detail" target="_blank" class="CoveringLink-a3s3kt-0 dXJclF"></a></div></div></div><div class="sc-cvbbAY kCbWgs"><div data-testid="gsl.uilib.Slider.Item[0]" direction="0" class="sc-eHgmQL cPQjPr" style="left: calc(0% + 0px);"><div><div class="Card__Photo-sc-7insep-2 dIgcyT" style="background-image: url(&quot;https://v.seloger.com/s/width/400/visuels/2/3/5/g/235gnqbocdzimv1umgyweg4h73wfsp441896ertb3.jpg&quot;);"></div><a z="1" href="https://www.seloger.com/annonces/achat/appartement/vincennes-94/162414425.htm?projects=2&amp;types=1&amp;natures=1,2&amp;places=[{cp:75}|{cp:93}|{cp:91}|{cp:94}|{cp:92}]&amp;rooms=1,2,3&amp;enterprise=0&amp;qsVersion=1.0&amp;m=search_to_detail" target="_blank" class="CoveringLink-a3s3kt-0 dXJclF"></a></div></div><div data-testid="gsl.uilib.Slider.Item[1]" direction="1" class="sc-eHgmQL cPQjPr" style="left: calc(100% + 0px);"><div><div class="Card__Photo-sc-7insep-2 dIgcyT" style="background-image: url(&quot;https://v.seloger.com/s/width/400/visuels/2/0/1/2/201279niip8kdlxz6s8nc61b19586u3okz4fz1ycf.jpg&quot;);"></div><a z="1" href="https://www.seloger.com/annonces/achat/appartement/vincennes-94/162414425.htm?projects=2&amp;types=1&amp;natures=1,2&amp;places=[{cp:75}|{cp:93}|{cp:91}|{cp:94}|{cp:92}]&amp;rooms=1,2,3&amp;enterprise=0&amp;qsVersion=1.0&amp;m=search_to_detail" target="_blank" class="CoveringLink-a3s3kt-0 dXJclF"></a></div></div></div><button data-testid="gsl.uilib.Slider.NextButton" direction="next" class="sc-hSdWYo goAGVH"><span><div data-testid="gsl.uilib.Icon" class="sc-bdVaJa hCkRUj"><svg width="1em" height="1em" viewBox="0 0 20 20" role="img"><defs><path d="M5.954 19a.94.94 0 01-.87-.576.947.947 0 01.194-1.029l7.11-7.39-7.11-7.39A.946.946 0 015.96 1a.937.937 0 01.67.303L15 10.005l-8.37 8.702a.937.937 0 01-.676.293z" id="chevron-right-bold_svg__a"></path></defs><use fill="currentColor" xlink:href="#chevron-right-bold_svg__a" fill-rule="evenodd"></use></svg></div></span></button></div></div><div type="video" class="MediaZone__RichMediaLayer-bd1uhq-0 fSKSXu"></div><div class="MediaZone__Stickers-bd1uhq-3 iwwrzY"></div><div class="MediaZone__Chips-bd1uhq-4 ifvlFB"><div class="MediaZone__Chip-bd1uhq-5 eUGAIh"><div data-test="sl-ui.icon" color="white" style="margin-right:6px" class="Icon__IconElement-sc-1xnn3zd-0 iIyIAR"><svg viewBox="0 0 512 512"><path d="M421.5 136h-50.3c-.9-.2-1.7-.7-2.4-1.4l-13-37.6c-4.3-12.4-17.5-21.7-30.7-21.7h-143c-13.3 0-26.6 9.3-30.8 21.6l-13 37.5c-.6.8-1.4 1.3-2.3 1.6H85.6c-31.5 0-57 24.9-57 55.5v182.1c0 30.6 25.6 55.5 57 55.5h335.9c31.5 0 57.1-24.9 57.1-55.5V191.5c-.1-30.6-25.6-55.5-57.1-55.5zm26.9 237.6c0 14-12.1 25.4-26.9 25.4H85.6c-14.8 0-26.9-11.4-26.9-25.4V191.5c0-14 12.1-25.4 26.9-25.4H136c13.3 0 26.6-9.3 30.8-21.7l12.9-37.4c.6-.8 1.5-1.4 2.4-1.6h142.7c.9.2 1.7.7 2.4 1.4l13 37.6c4.2 12.4 17.5 21.7 30.8 21.7h50.4c14.9 0 26.9 11.4 26.9 25.4l.1 182.1z"></path><path d="M253.5 180.3c-54.7 0-99 44.3-99 99s44.3 99 99 99 99-44.3 99-98.9c0-54.7-44.3-99-99-99.1zm0 168c-38.1 0-68.9-30.8-68.9-68.9s30.8-68.9 68.9-68.9 68.9 30.8 68.9 68.9c0 38-30.8 68.8-68.9 68.9zM396.2 180.3c-17.6 0-31.8 14.2-31.8 31.8s14.2 31.8 31.8 31.8 31.8-14.2 31.8-31.8-14.2-31.8-31.8-31.8z"></path></svg></div>4</div><div class="MediaZone__Chip-bd1uhq-5 eUGAIh">Vidéo</div></div></div><div class="Card__ContentZone-sc-7insep-3 gfORyM"><div class="ContentZone__DetailTop-wghbmy-6 dPYwvZ"><div><div class="ContentZone__Title-wghbmy-7 joPkKZ">Appartement</div><ul class="ContentZone__Tags-wghbmy-8 eJYQQA"><li>3 p</li><li>2 ch</li><li>45 m²</li></ul></div><div class="Price__PriceContainer-sc-1g9fitq-0 bxGeRg"><div class="Price__Label-sc-1g9fitq-1 mVWFG">438&nbsp;000 €</div></div></div><div class="ContentZone__Address-wghbmy-2 fRAjHL"><span>Vincennes</span></div><div class="Card__Description-sc-7insep-4 ContentZone__ClassifiedDescription-wghbmy-0 bYHXHs">Vincennes Coeur de Ville - Proche de la Mairie, CHANELLE IMMOBILER vous propose ce Ravissant 2/3 Pièces en Etage. Il se compo</div><div class="ContentZone__ButtonSection-wghbmy-4 fgfCuS"><div class="ContentZone__Button-wghbmy-5 ePrJwG"><div data-test="sl-ui.icon" class="ContentZone__StyledIcon-wghbmy-3 jIMdci Icon__IconElement-sc-1xnn3zd-0 jGTLkV"><svg width="1em" height="1em" viewBox="0 0 20 20"><path d="M18.146 2.146a.5.5 0 01.708.707l-16 16a.5.5 0 11-.708-.707l3.728-3.728c-1.421-.893-2.696-2.106-3.782-3.643a.503.503 0 01-.003-.574c.15-.215 3.718-5.277 8.886-5.188.018-.003 1.388-.088 3.335.97zm-1.794 5.258c.88.76 1.739 1.697 2.549 2.784a.5.5 0 01-.001.598c-.161.216-3.938 5.204-8.231 5.204l-.186-.003h-.034c-.148-.002-.776-.029-1.666-.265a.5.5 0 01.256-.967c.895.238 1.472.237 1.456.232 3.276.107 6.5-3.45 7.372-4.507a16.03 16.03 0 00-2.168-2.32.5.5 0 01.653-.756zm-2.26 2.226c.075.281.113.571.113.862 0 1.935-1.658 3.508-3.697 3.508-.22 0-.433-.018-.643-.053a.5.5 0 11.164-.986c.156.025.316.039.479.039 1.487 0 2.697-1.125 2.697-2.508 0-.205-.026-.408-.08-.605a.501.501 0 01.968-.257zM10.95 6.01h-.054c-3.958 0-6.971 3.461-7.774 4.482 1.01 1.36 2.179 2.43 3.473 3.206l1.033-1.033c-.521-.618-.817-1.37-.817-2.173 0-1.935 1.658-3.508 3.697-3.508a3.83 3.83 0 012.138.662l.927-.926c-1.525-.764-2.553-.708-2.568-.709h-.109zm-.442 1.974c-1.488 0-2.697 1.125-2.697 2.508 0 .535.187 1.042.522 1.468l3.595-3.595a2.835 2.835 0 00-1.42-.38z" fill="currentColor" fill-rule="evenodd"></path></svg></div><span>Masquer</span></div><div class="ContentZone__Button-wghbmy-5 dxnwuI"><div data-test="sl-ui.icon" class="ContentZone__StyledIcon-wghbmy-3 jIMdci Icon__IconElement-sc-1xnn3zd-0 jGTLkV"><svg width="1em" height="1em" viewBox="0 0 20 20"><defs><path d="M9.976 18.5l-.706-.484c-5.564-3.54-8.64-7.87-8.234-11.568.13-2.704 2.267-4.854 4.915-4.944a4.9 4.9 0 014.025 1.84A5.014 5.014 0 0114 1.513c2.67.1 4.825 2.266 4.963 4.993.407 3.609-2.69 7.91-8.282 11.578l-.706.415zM5.95 3.017c-1.888.1-3.391 1.65-3.474 3.58-.338 3.123 2.468 6.9 7.499 10.143 4.963-3.243 7.827-7.099 7.498-10.065-.058-1.972-1.593-3.566-3.521-3.658a3.576 3.576 0 00-3.397 1.977L9.918 6.22l-.639-1.226a3.51 3.51 0 00-3.328-1.977z" id="heart-regular_svg__a"></path></defs><use fill="currentColor" xlink:href="#heart-regular_svg__a" fill-rule="evenodd"></use></svg></div><span>Sauvegarder</span></div></div><div mode="big" class="ContentZone__SmartClassifiedContact-wghbmy-1 bqeKmt Contact__ContactContainer-sc-3d01ca-1 jMFCBR"><a href="http://www.chanelleimmobilier.com" rel="nofollow" target="_blank" class="Contact__WebSiteShortcutContainer-sc-3d01ca-0 csZlpz"><div data-test="icon" color="#929292" class="Icon__IconElement-sc-1xnn3zd-0 qqhlj"><svg viewBox="0 0 512 512"><path d="M478.2 254.2c0-120.6-93.3-219.3-211.2-226.5-1.2-.3-2.5-.4-3.8-.4-1.3 0-2.7 0-4 .1-2-.1-4-.1-6.1-.1h-4.2C149.1 29.2 65.1 96.7 37.7 188.9c-.8 1.7-1.4 3.6-1.6 5.5-5.3 19.5-7.9 39.6-7.9 59.8 0 123.9 98.5 224.7 220.9 226.9h4.1c2.1 0 4.1 0 6.1-.1 1.3 0 2.6.1 3.9.1 1.3 0 2.5-.1 3.8-.4 117.9-7.2 211.2-105.8 211.2-226.5zm-416.2 0c0-13.8 1.5-27.6 4.4-41.2h56.5c-3 26.2-3.1 52.6-.4 78.8H65.7c-2.5-12.3-3.7-24.9-3.7-37.6zm92.3 0c0-13.8.8-27.5 2.6-41.2h79.5v78.8h-79.8c-1.6-12.4-2.3-25-2.3-37.6zm196-75h-80.2V64.5c35.4 12.2 65.9 55.8 80.2 114.7zm-114-112v112H163c13.4-55 41-96.7 73.3-112zm0 258.4v115.5c-33.1-15.5-61.1-58.6-74.1-115.5h74.1zm33.8 118.3V325.7h81c-13.9 60.7-44.9 105.8-81 118.2zm0-152v-78.8h86.3c1.7 13.7 2.6 27.4 2.6 41.2 0 12.6-.7 25.2-2.1 37.7l-86.8-.1zM390.5 213H440c5.6 26 5.8 52.8.7 78.8h-49.9c1.3-12.5 1.9-25.1 1.9-37.7.1-13.7-.7-27.4-2.2-41.1zm39-33.8H385c-7.9-36.4-21.3-68.6-39-93.9 37.6 21.1 66.9 54.2 83.5 93.9zm-258-99.6c-19.6 26.1-34.6 60.5-43.1 99.7H77c18.2-43.7 51.8-79.2 94.5-99.7zM75.6 325.7h52.1c8.3 40.6 23.6 76.3 43.8 103.2-43.9-21.1-78.1-58-95.9-103.2zM346 423.1c18.2-26.1 32-59.7 39.7-97.5h45.2c-16.3 41.4-46.2 75.8-84.9 97.5z"></path></svg></div></a><a class="AgencyLogo__AgencyLogoContainer-fcobow-0 jgWjpP" mode="big" target="_blank" href="https://www.seloger.com/professionnels/agences-immobilieres/vincennes-94/agency-2886828.htm?bd=Liste_annonce_annuaire_detail"><div class="AgencyLogo__Placeholder-fcobow-2 jRuXFY">AGENCE CHANELLE IMMOBILIER</div></a><div mode="big" class="Contact__Buttons-sc-3d01ca-3 brnQTP"><div data-test="button-container" class="Contact__ButtonContainer-sc-3d01ca-2 iklMcR"><div data-test="icon" class="Contact__ButtonIcon-sc-3d01ca-6 hWcXVi Icon__IconElement-sc-1xnn3zd-0 jGTLkV"><svg viewBox="0 0 512 512"><g><path d="M391.3 382.7c-68.4-69.2-73.8-72.7-76.6-74.4-12.7-7.9-26.2-6.8-40.3 3.3-3 2.2-16.9 12.7-23.3 27.3-1.3-.6-2.5-1.2-3.7-1.9-12.6-7.3-25.8-17.9-39.3-31.6l-3.1-3.1c-13.5-13.7-23.9-27.1-30.9-39.7-.7-1.2-1.3-2.5-1.8-3.8 14.5-6.1 25.4-19.9 27.6-22.8 10.5-13.9 11.9-27.4 4.1-40.4-1.6-2.7-5-8.2-73.2-77.7-7.9-8.2-18.9-12.8-30.3-12.6-9.8.5-19.2 4.2-26.7 10.7-19.5 16.4-29.9 38.1-35.2 53.5-4.3 12.6-6.8 25.7-7.6 39-4.8 78.3 41.7 142.9 81.5 183.4l.1.1 4.3 4.4c38.4 39.1 99.3 84.7 173.9 84.7 2.7 0 5.4-.1 8.1-.2 13.3-.5 26.5-2.8 39.2-6.9 15.5-5 37.5-15 54.2-34.3 6.8-7.9 10.6-17 11.1-26.4.4-11.5-4-22.5-12.1-30.6.1 0 0 0 0 0zm-205.6-55.4c15.5 15.7 31 28.1 46 36.8 5.9 3.6 12.5 6 19.3 7.1 8.7 1.3 13.7-.9 17.5-3 5.3-3.2 9-8.6 10.9-15.8 1.1-4.4 7.8-11.5 13.3-15.5 1-.7 2.3-2.5 4.9-2.5 1.8 1.4 12.3 10.4 71.3 70.1 2 1.8 3.1 4.4 3.1 7.1-.1 2.3-1.3 4.9-3.5 7.4-12 13.8-28.4 21.2-40.2 25-10 3.2-20.4 5-30.9 5.3-67.9 3-123-39.2-158.4-75.3l-2.1-2.1c-.8-.8-1.6-1.5-2.3-2.2l.1-.1C99.4 334 58.1 277.1 62.2 210.4c.6-10.5 2.6-20.8 5.9-30.8 4-11.6 11.7-27.9 25.7-39.6 2.6-2.2 5.1-3.3 7.5-3.4h.2c2.6.1 5.1 1.3 6.9 3.2 60.2 61.4 68 71.1 69.1 72.8-.1 1-1.9 3.4-2.7 4.5-4 5.4-11.2 11.9-15.6 13-7.3 1.7-12.7 5.3-16.3 10.9-1.9 3.1-4.3 8.2-3.1 17.2 1 6.8 3.3 13.4 6.8 19.4 8.4 15.1 20.5 30.8 36 46.5l3.1 3.2zM239.1 105.8c-8.5-.9-16.1 5.3-17 13.7v.5c-.8 8.6 5.5 16.2 14.1 17h.1c73 6.5 131 64.1 138 137 .8 8 7.5 14.2 15.6 14.2.5 0 1 0 1.5-.1 8.6-.8 14.9-8.5 14.1-17.1-8.6-87.9-78.5-157.3-166.4-165.2z"></path><path d="M407.5 104.2c-42-42-97.4-67.9-156.6-73.2-8.6-.8-16.2 5.6-17 14.2s5.6 16.2 14.2 17c106.4 9.5 191 93.4 201.3 199.8.8 8 7.5 14.2 15.6 14.2.5 0 1 0 1.5-.1 8.6-.8 14.9-8.5 14.1-17.1-5.7-58.5-31.6-113.2-73.1-154.8z"></path></g></svg></div></div><div data-test="button-container" class="Contact__ButtonContainer-sc-3d01ca-2 iklMcR"><div data-test="icon" class="Contact__ButtonIcon-sc-3d01ca-6 hWcXVi Icon__IconElement-sc-1xnn3zd-0 jGTLkV"><svg viewBox="0 0 512 512"><path d="M449.07 431.55H64.33c-29.06 0-52.63-24.37-52.63-54.44v-245c0-30.07 23.57-54.44 52.63-54.44h384.74c29.07 0 52.63 24.37 52.63 54.44v245c0 30.07-23.56 54.44-52.63 54.44zm12.37-281.73L256.7 354.24 52 149.59v229.7a12.58 12.58 0 0 0 12.37 12.52h384.7a12.58 12.58 0 0 0 12.37-12.52zm-380.8-28.33L256.7 297.35 433 121.49z" data-name="R/uEA51-B-enveloppe"></path></svg></div></div></div></div><a name="classified-link" z="1" href="https://www.seloger.com/annonces/achat/appartement/vincennes-94/162414425.htm?projects=2&amp;types=1&amp;natures=1,2&amp;places=[{cp:75}|{cp:93}|{cp:91}|{cp:94}|{cp:92}]&amp;rooms=1,2,3&amp;enterprise=0&amp;qsVersion=1.0&amp;m=search_to_detail" target="" class="CoveringLink-a3s3kt-0 dXJclF"></a></div></div>